# cert-manager

---

Installation

In [ ]:
%%bash
microk8s enable cert-manager

### CA für interne Zertifikate einrichten
Da es sich um eine interne DNS-Domain handelt, kann Let's Encrypt nicht genutzt werden. Stattdessen gibt es zwei Optionen:

Option 1: Self-Signed Issuer (einfach)
Falls du nur ein selbstsigniertes Zertifikat für interne Zwecke benötigst:

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: cert-manager.io/v1
kind: Issuer
metadata:
  name: selfsigned-issuer
  namespace: default
spec:
  selfSigned: {}
EOF


### Zertifikatsanfrage für die interne Domain erstellen

Jetzt kann ein Zertifikat für die interne DNS-Domain (cna-cloud-hf-08-cp1.maas) erstellt werden:

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: cert-manager.io/v1
kind: Certificate
metadata:
  name: internal-cert
  namespace: default
spec:
  secretName: internal-cert-secret
  issuerRef:
    name: selfsigned-issuer
    kind: ClusterIssuer
  commonName: cna-cloud-hf-08-cp1.maas
  dnsNames:
    - cna-cloud-hf-08-cp1.maas
EOF

### Verwendung des Zertifikats

Das Zertifikat liegt nun als Kubernetes-Secret unter internal-cert-secret und kann z. B. für Ingress oder TLS-Services genutzt werden

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: internal-ingress
  annotations:
    cert-manager.io/cluster-issuer: selfsigned-issuer
spec:
  tls:
    - hosts:
        - cna-cloud-hf-08-cp1.maas
      secretName: internal-cert-7797l
  rules:
    - host: cna-cloud-hf-08-cp1.maas
      http:
        paths:
          - path: /
            pathType: Prefix
            backend:
              service:
                name: my-service
                port:
                  number: 80
                port:
                  number: 443                  
EOF


In [ ]:
%%bash
kubectl describe certificate internal-cert -n default
kubectl get secret internal-cert-7797l -n default
